In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('C:\\Users\\HP\\Desktop\\4.sınıf Güz Dönemi\\Profesyonel Yazılım Geliştirme\Profesyonel Yazılım Geliştirme 2\\arabam.com\\Dataset1-4k\\cars-fill-training.csv')

In [3]:
dataset.head()

,fuel,year,selling_price,km_driven,seller_type,transmission,owner
0,1,2007,60000,70000,1,1,1
1,1,2007,135000,50000,1,1,1
2,0,2012,600000,100000,1,1,0
3,1,2017,250000,46000,1,1,1
4,0,2014,450000,141000,1,1,0


In [4]:
#price kolonu dataFrame'in en sonuna eklendi
price = dataset["selling_price"]
dataset.drop("selling_price",axis=1, inplace=True)
dataset["selling_price"] = price

In [5]:
dataset

,fuel,year,km_driven,seller_type,transmission,owner,selling_price
0,1,2007,70000,1,1,1,60000
1,1,2007,50000,1,1,1,135000
2,0,2012,100000,1,1,0,600000
3,1,2017,46000,1,1,1,250000
4,0,2014,141000,1,1,0,450000
...,...,...,...,...,...,...,...
4271,0,2014,80000,1,1,0,409999
4272,0,2014,80000,1,1,0,409999
4273,1,2009,83000,1,1,1,110000
4274,0,2016,90000,1,1,0,865000


In [6]:
X = dataset.iloc[:, 0:6].values
y = dataset.iloc[:, 6].values
y = y.reshape(-1, 1) 

In [7]:
X

array([[     1,   2007,  70000,      1,      1,      1],
       [     1,   2007,  50000,      1,      1,      1],
       [     0,   2012, 100000,      1,      1,      0],
       ...,
       [     1,   2009,  83000,      1,      1,      1],
       [     0,   2016,  90000,      1,      1,      0],
       [     1,   2016,  40000,      1,      1,      1]], dtype=int64)

In [8]:
y

array([[ 60000],
       [135000],
       [600000],
       ...,
       [110000],
       [865000],
       [225000]], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=0)

In [10]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import chi2


# feature selection
def select_features(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=mutual_info_regression, k=3)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Feature 0: 0.120709
Feature 1: 0.397572
Feature 2: 0.094996
Feature 3: 0.153948
Feature 4: 0.126186
Feature 5: 0.097300


### Decision Tree Regressor

In [11]:
from sklearn.tree import DecisionTreeRegressor
regressorFS = DecisionTreeRegressor(random_state = 0)
regressorFS.fit(X_train_fs, y_train)
y_predDTfs = regressorFS.predict(X_test_fs)

In [12]:
from sklearn.metrics import r2_score
DT_RSCOREfs = r2_score(y_test, y_predDTfs) 

In [19]:
#DecisionTreeRegressor algoritması parametresiz çalıştırıldığında aldığı değer
DT_RSCOREfs

0.48967586612717606

In [20]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(criterion="absolute_error", splitter="random",min_samples_split=7,random_state = 0)
regressor.fit(X_train, y_train)
y_predDT = regressor.predict(X_test)

In [21]:
from sklearn.metrics import r2_score
DT_RSCORE = r2_score(y_test, y_predDT) 

In [22]:
#DecisionTreeRegressor algoritması için parameter tuning yapılması durumunda ulaşılan en yüksek başarım oranı
print(DT_RSCORE)

0.5344976085229363


In [23]:
import joblib
joblib.dump(regressor, "../Models/cars_decision_tree_model.pkl")

['../Models/cars_decision_tree_model.pkl']

### Random Forest Regressor with RandomizedSearchCV

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score

rf_base_model = RandomForestRegressor()
rf_base_model.fit(X_train,y_train)
y_pred  = rf_base_model.predict(X_test)
RF_RSCORE= r2_score(y_test,y_pred)
print('rf base model r2_score= {}'.format(RF_RSCORE))

C:\Users\HP\AppData\Local\Temp\ipykernel_13020\1020849373.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_base_model.fit(X_train,y_train)


rf base model r2_score= 0.5869644499169387


In [25]:
# RandomizeSearchCV

rf_params = {'criterion': ["squared_error", "absolute_error", "poisson"],
    'max_depth': [80, 90, 100, 110],
    'max_features': ["auto", "sqrt", "log2"],
    'min_samples_leaf': range(1,11),
    'min_samples_split': range(1,15,5),
    'n_estimators': [100, 200, 300,500,1000]}

rf = RandomForestRegressor()
rf_randomcv_model = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, n_iter=200, cv=5, n_jobs=-1, verbose=2).fit(X_train,y_train)
rf_randomcv_model.best_params_
print('rf randomcv model r2_score = {}'.format(rf_randomcv_model.best_score_))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
280 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\ProgramData\Anaconda3\lib\site-package

rf randomcv model r2_score = 0.6716447154070387


In [27]:
import joblib
joblib.dump(rf_randomcv_model, "../Models/cars_random_forest_model.pkl")

['../Models/cars_random_forest_model.pkl']

### LinearRegression with RandomizedCV

In [28]:
from sklearn.linear_model import LinearRegression

lr_base_model = LinearRegression()
lr_base_model.fit(X_train,y_train)
y_pred  = lr_base_model.predict(X_test)
LR_RSCORE= r2_score(y_test,y_pred)
print('lr base model r2_score= {}'.format(LR_RSCORE))

lr base model r2_score= 0.425960272966426


In [34]:
import joblib
joblib.dump(regressor, "../Models/cars_linear_regression_model.pkl")

['../Models/cars_linear_regression_model.pkl']

### SVR with RandomizedCV 

In [38]:
"""
SVR veri seti üzerinde train edildi. Ancak yetersiz başarım oranından dolayı kullanılmayacak.
"""
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

svr_base_model = SVC()
svr_base_model.fit(X_train,y_train)
y_pred  = svr_base_model.predict(X_test)
SVR_RSCORE= r2_score(y_test,y_pred)
print('svr base model r2_score= {}'.format(SVR_RSCORE))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svr base model r2_score= -0.11346083907173887
